## **ESTRATÉGIA DE CUPONS NA RETENÇÃO DE USUÁRIOS**

### BIBLIOTECAS

In [0]:
import numpy as np
import pandas as pd
from scipy import stats
import plotly.express as px
import matplotlib.pyplot as plt
from pyspark.ml import Pipeline
import plotly.graph_objects as go
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from scipy.stats import chi2_contingency
from pyspark.ml.clustering import KMeans
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.sql.functions import sum as spark_sum
from pyspark.sql.functions import col, count, sum, avg, stddev, min, max, countDistinct, datediff, from_unixtime, to_date, when, rank, dense_rank, percent_rank, ntile, lag, explode, split, year, month, dayofweek, col, current_date, datediff, to_timestamp, col, when, desc, avg, count, sum, round as spark_round


### EXTRAÇÃO

In [0]:
def import_s3_data(spark: SparkSession, s3_repositories: Dict[str, str]) -> Dict[str, DataFrame]:

    dataframes = {}
    
    for name, path in s3_repositories.items():
        try:
            # Converter caminhos HTTPS S3 para o protocolo s3a
            if path.startswith('https://'):
                # Extrair bucket e chave da URL
                path_parts = path.replace('https://', '').split('/', 1)
                bucket_parts = path_parts[0].split('.')
                bucket = bucket_parts[0]
                key = path_parts[1] if len(path_parts) > 1 else ''
                s3_path = f"s3a://{bucket}/{key}"
            else:
                s3_path = path
            
            # Detectar tipo de arquivo e compactação
            file_extension = path.lower().split('.')[-1]
            is_compressed = path.lower().endswith('.gz')
            file_type = path.lower().split('.')[-2] if is_compressed else file_extension
            
            # Leia os dados com base no tipo de arquivo
            if 'json' in file_type:
                df = spark.read.option("compression", "gzip").json(s3_path)
            elif 'csv' in file_type:
                df = spark.read.option("compression", "gzip").option("header", "true").option("inferSchema", "true").csv(s3_path)
            else:
                print(f"Aviso: Tipo de arquivo desconhecido para {name}.")
                continue
                
            dataframes[name] = df
            print(f"Conjunto de dados {name} importado")
            
        except Exception as e:
            print(f"Erro ao importar {name}: {str(e)}")
    
    return dataframes

# Definindo os repositórios S3
repositories = {
    "orders": "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz",
    "consumers": "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz",
    "restaurants": "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz",
    # "ab_test_ref": "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"
}

# Importando os dados (exclui automaticamente o arquivo tar.gz)
dataframes = import_s3_data(spark, repositories)

Conjunto de dados orders importado
Conjunto de dados consumers importado
Conjunto de dados restaurants importado


#### Orders


- cpf (string): Cadastro de Pessoa Física do usuário que realizou o pedi do  
- customer_id (string): Identificador do usuário  
- customer_name (string): Primeiro nome do usuário  
- delivery_address_city (string): Cidade de entrega do pedido  - delivery_address_country (string): País da entrega  
- delivery_address_district (string): Bairro da entrega  
- delivery_address_external_id (string): Identificador do endereço  de entrega  
- delivery_address_latitude (float): Latitude do endereço de entre ga  a
- delivery_address_longitude (float): Longitude do endereço de  entrega  
- delivery_address_state (string): Estado da entrega  
- delivery_address_zip_code (string): CEP da entrega  
- items (array[json]): Itens que compõem o pedido, bem como informa ções complementares como preço unitário, quantidade, etc.
- merchant_id (string): Identificador do restaurante  
- merchant_latitude (float): Latitude do restaurante  
- merchant_longitude (float): Longitude do reslinestringtaurante  
- merchant_timezone (string): Fuso horário em que o restaurante está  localizado  
- order_created_at (timestamp): Data e hora em que o pedido foi cri ado  
- order_id (string): Identificador do pedido  
- order_scheduled (bool): Flag indicando se o pedido foi agendado ou  não (pedidos agendados são aqueles que o usuário escolheu uma data  e hora para a entrega)  
- order_total_amount (float): Valor total do pedido em Reais  - origin_platform (string): Sistema operacional do dispositivo do  usuário  
- order_scheduled_date (timestamp): Data e horário para entrega do  pedido agendado  


In [0]:
# Criando Dataframe
orders_df = dataframes["orders"]

orders_df.limit(5).toPandas()

,cpf,customer_id,customer_name,delivery_address_city,delivery_address_country,delivery_address_district,delivery_address_external_id,delivery_address_latitude,delivery_address_longitude,delivery_address_state,delivery_address_zip_code,items,merchant_id,merchant_latitude,merchant_longitude,merchant_timezone,order_created_at,order_id,order_scheduled,order_scheduled_date,order_total_amount,origin_platform
0,80532101763,7ba88a68bb2a3504c6bd37a707af57a0b8d6e110a551c7...,GUSTAVO,FRANCA,BR,JARDIM ESPRAIADO,6736655,-47.39,-20.55,SP,14403,"[{""name"": ""Parmegiana de Filé de Frango (2 pes...",a992a079a651e699d9149423761df2427c0e3af0a2a1b5...,-47.39,-20.55,America/Sao_Paulo,2019-01-17T22:50:06.000Z,33e0612d62e5eb42aba15b58413137e441fbe906de2feb...,False,None,46.0,ANDROID
1,43352103961,078acecdcf7fa89d356bfa349f14a8219db1ee161ce28a...,MICHELLE,SANTOS,BR,CAMPO GRANDE,8759216,-46.34,-23.96,SP,11070,"[{""name"": ""Filé Mignon à Cubana"", ""addition"": ...",5152f28ee0518b8803ccf0a4096eb2ff8b81e9491861c9...,-46.34,-23.96,America/Sao_Paulo,2019-01-17T17:51:26.000Z,148c4353a2952f3fe7973547283265eb22b575fb712ed2...,False,None,104.5,ANDROID
2,38650991217,0e38a3237b5946e8ab2367b4f1a3ae6e77f1e215bc760c...,VICTOR,GUARULHOS,BR,JARDIM ROSSI,8765930,-46.53,-23.44,SP,71304,"[{""name"": ""GRANDE 2 SABORES"", ""addition"": {""va...",b6096419455c35d06105a5ef0d25c51f9dd40e1e99ac33...,-46.53,-23.44,America/Sao_Paulo,2019-01-17T22:53:47.000Z,c37e495a91b498bb7b70a9e09ac115d0cdd443f152dc11...,False,None,35.0,IOS
3,63579726866,cab1a004b7206d07910092c515a79834fea0a03d7d9054...,ANNIE,SAO PAULO,BR,PARQUE SAO JORGE,7834087,-46.57,-23.53,SP,30870,"[{""name"": ""CALABRESA"", ""addition"": {""value"": ""...",082bfdcdf6ccdc343e3c4d25ee376b5b6ca7e96ad8b04e...,-46.57,-23.53,America/Sao_Paulo,2019-01-17T23:56:53.000Z,b4df94142d21354611247da9ca94f870c09b93989b531a...,False,None,40.8,IOS
4,90617788806,aa7edf5b166b8c843aec3b96dc561222888734f3879123...,DANIEL,VITORIA,BR,JARDIM CAMBURI,7211683,-40.27,-20.25,ES,29090,"[{""name"": ""GRANDE (35CM) 8 PDÇ 2 SABORES"", ""ad...",d7adb764bac29ccb77fb8f746ffbd531bf05ec30a7e130...,-40.27,-20.25,America/Sao_Paulo,2019-01-17T23:40:53.000Z,4ff64b33b272c1886df21b63272220af6a82d1667dba70...,False,None,48.5,ANDROID


In [0]:
# Contar o total de linhas do DataFrame
orders_df_count = orders_df.count()
print(f"Total de linhas: {orders_df_count}")

# Verificando se há customer_ids duplicados
duplicate_orders_count = orders_df_count - orders_df.dropDuplicates(["customer_id"]).count()
print(f"Número de customer_ids duplicados: {duplicate_orders_count}")

Total de linhas: 2432974
Número de customer_ids duplicados: 1626507


In [0]:
# Selecionar somente as colunas necessárias
orders_df = orders_df[["customer_id", 
        "delivery_address_city", 
        "delivery_address_state",
        "merchant_id", 
        "order_id",
        "origin_platform",
        "order_scheduled",
        "order_total_amount"]]

# Remover os duplicados
orders_df = orders_df.dropDuplicates()

# Contar o total de linhas do DataFrame
orders_df_count = orders_df.count()
print(f"Total de linhas: {orders_df_count}")

Total de linhas: 2432974


In [0]:
# Verificando as primeiras linhas
orders_df.limit(5).toPandas()

,customer_id,delivery_address_city,delivery_address_state,merchant_id,order_id,origin_platform,order_scheduled,order_total_amount
0,aa7edf5b166b8c843aec3b96dc561222888734f3879123...,VITORIA,ES,d7adb764bac29ccb77fb8f746ffbd531bf05ec30a7e130...,4ff64b33b272c1886df21b63272220af6a82d1667dba70...,ANDROID,False,48.5
1,078acecdcf7fa89d356bfa349f14a8219db1ee161ce28a...,SANTOS,SP,5152f28ee0518b8803ccf0a4096eb2ff8b81e9491861c9...,148c4353a2952f3fe7973547283265eb22b575fb712ed2...,ANDROID,False,104.5
2,7ba88a68bb2a3504c6bd37a707af57a0b8d6e110a551c7...,FRANCA,SP,a992a079a651e699d9149423761df2427c0e3af0a2a1b5...,33e0612d62e5eb42aba15b58413137e441fbe906de2feb...,ANDROID,False,46.0
3,0e38a3237b5946e8ab2367b4f1a3ae6e77f1e215bc760c...,GUARULHOS,SP,b6096419455c35d06105a5ef0d25c51f9dd40e1e99ac33...,c37e495a91b498bb7b70a9e09ac115d0cdd443f152dc11...,IOS,False,35.0
4,cab1a004b7206d07910092c515a79834fea0a03d7d9054...,SAO PAULO,SP,082bfdcdf6ccdc343e3c4d25ee376b5b6ca7e96ad8b04e...,b4df94142d21354611247da9ca94f870c09b93989b531a...,IOS,False,40.8


#### Restaurants

- id (string): Identificador do restaurante  
- created_at (timestamp): Data e hora em que o restaurante foi criado  - enabled (bool): Flag indicando se o restaurante está ativo no iFood ou  não  
- price_range (int): Classificação de preço do restaurante  
- average_ticket (float): Ticket médio dos pedidos no restaurante  - delivery_time (float): Tempo padrão de entrega para pedidos no restaurante  
- minimum_order_value (float): Valor mínimo para pedidos no restau rante  
- merchant_zip_code (string): CEP do restaurante  
- merchant_city (string): Cidade do restaurante  
- merchant_state (string): Estado do restaurante  
- merchant_country (string): País do restaurante

In [0]:
# Criando Dataframe
restaurants_df = dataframes["restaurants"]

restaurants_df.limit(5).toPandas()

,id,created_at,enabled,price_range,average_ticket,takeout_time,delivery_time,minimum_order_value,merchant_zip_code,merchant_city,merchant_state,merchant_country
0,d19ff6fca6288939bff073ad0a119d25c0365c407e9e5d...,2017-01-23 12:52:30.910,False,3,60.0,0,50,30.0,14025,RIBEIRAO PRETO,SP,BR
1,631df0985fdbbaf27b9b031a8f381924e3483833385748...,2017-01-20 13:14:48.286,True,3,60.0,0,0,30.0,50180,SAO PAULO,SP,BR
2,135c5c4ae4c1ec1fdc23e8c649f313e39be8db913d8bc5...,2017-01-23 12:46:33.457,True,5,100.0,0,45,10.0,23090,RIO DE JANEIRO,RJ,BR
3,d26f84c470451f752bef036c55517b6d9950d41806f10e...,2017-01-20 13:15:04.806,True,3,80.0,0,0,18.9,40255,SALVADOR,BA,BR
4,97b9884600ea7192314580d9115f8882b8634f5aa201ff...,2017-01-20 13:14:27.701,True,3,60.0,0,0,25.0,64600,BARUERI,SP,BR


In [0]:
# Verificando quantidade de linhas
restaurants_df_count = restaurants_df.count()
print(f"Total de linhas: {restaurants_df_count}")

# Verificando se há customer_ids duplicados
duplicate_restaurants_count = restaurants_df_count - restaurants_df.dropDuplicates(["id"]).count()
print(f"Número de id_restaurants duplicados: {duplicate_restaurants_count}")

Total de linhas: 7292
Número de id_restaurants duplicados: 0


#### Consumers

- customer_id (string): Identificador do usuário  
- language (string): Idioma do usuário  
- created_at (timestamp): Data e hora em que o usuário foi criado  - active (bool): Flag indicando se o usuário está ativo ou não 
- customer_name (string): Primeiro nome do usuário  
- customer_phone_area (string): Código de área do telefone do usuá rio  
- customer_phone_number (string): Número do telefone do usuário  

In [0]:
consumers_df = dataframes["consumers"]

consumers_df.limit(5).toPandas()

,customer_id,language,created_at,active,customer_name,customer_phone_area,customer_phone_number
0,e8cc60860e09c0bb19610b06ced69c973eb83982cfc98e...,pt-br,2018-04-05 14:49:18.165,True,NUNO,46,816135924
1,a2834a38a9876cf74e016524dd2e8c1f010ee12b2b684d...,pt-br,2018-01-14 21:40:02.141,True,ADRIELLY,59,231330577
2,41e1051728eba13341136d67d0757f8d8cc44b2a405b71...,pt-br,2018-01-07 03:47:15.554,True,PAULA,62,347597883
3,8e7c1dcb64edf95c935147f6d560cb068c44714cb1b21b...,pt-br,2018-01-10 22:17:08.160,True,HELTON,13,719366842
4,7823d4cf4150c5daeae0bd799206852fc123bdef0cf5d1...,pt-br,2018-04-06 00:16:20.935,True,WENDER,76,543232158


In [0]:
# Verificando quantidade de linhas
consumers_df_count = consumers_df.count()
print(f"Total de linhas: {consumers_df_count}")

# Verificando se há customer_ids duplicados
duplicate_consumers_count = consumers_df_count - consumers_df.dropDuplicates(["customer_id"]).count()
print(f"Número de customer_ids duplicados: {duplicate_consumers_count}")

Total de linhas: 806156
Número de customer_ids duplicados: 0


#### AB

- customer_id (string): Identificador do usuário  
- is_target (string): Grupo ao qual o usuário pertence ('target' ou  'control').

In [0]:
# Localização e tipo de arquivo
file_location = "/FileStore/tables/ab_test_ref-1.csv"
file_type = "csv"

# Opções de CSV
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# As opções aplicadas são para arquivos CSV. Para outros tipos de arquivo, elas serão ignoradas.
ab_test_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location) 

ab_test_df.limit(5).toPandas()

,customer_id,is_target
0,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,target
1,b821aa8372b8e5b82cdc283742757df8c45eecdd72adf4...,control
2,d425d6ee4c9d4e211b71da8fc60bf6c5336b2ea9af9cc0...,control
3,6a7089eea0a5dc294fbccd4fa24d0d84a90c1cc12e829c...,target
4,dad6b7e222bab31c0332b0ccd9fa5dbd147008facd268f...,control


In [0]:
# Transformando is_target em booleano
ab_test_df = ab_test_df.withColumn("is_target", F.when(F.col("is_target") == "target", True).otherwise(False))
ab_test_df.show(5)

+--------------------+---------+
|         customer_id|is_target|
+--------------------+---------+
|755e1fa18f25caec5...|     true|
|b821aa8372b8e5b82...|    false|
|d425d6ee4c9d4e211...|    false|
|6a7089eea0a5dc294...|     true|
|dad6b7e222bab31c0...|    false|
+--------------------+---------+
only showing top 5 rows



In [0]:
# Verificando quantidade de linhas
ab_test_df_count = ab_test_df.count()
print(f"Total de linhas: {ab_test_df_count}")

# Verificando se há customer_ids duplicados
duplicate_ab_count = ab_test_df_count - ab_test_df.dropDuplicates(["customer_id"]).count()
print(f"Número de customer_ids duplicados: {duplicate_ab_count}")

Total de linhas: 806467
Número de customer_ids duplicados: 0


In [0]:
participantes_acao = ab_test_df[ab_test_df['is_target'] == True].count()
print(participantes_acao)

445925


LEFT JOIN com a tabela consumers

In [0]:
# Testando LEFT JOIN com a tabela consumers
consumers_target_df = ab_test_df.join(consumers_df, on="customer_id", how="left")

# Convertendo a coluna 'created_at' para timestamp e calculando os dias ativos
consumers_target_df = consumers_target_df.withColumn(
    "created_at", to_timestamp(col("created_at"))
).withColumn(
    "days_active", datediff(current_date(), col("created_at"))
)

consumers_target_df.show(5)

+--------------------+---------+--------+--------------------+------+-------------+-------------------+---------------------+-----------+
|         customer_id|is_target|language|          created_at|active|customer_name|customer_phone_area|customer_phone_number|days_active|
+--------------------+---------+--------+--------------------+------+-------------+-------------------+---------------------+-----------+
|11eec65ca267ea72e...|     true|   pt-br|2018-04-05 15:01:...|  true|     CRISTINA|                 74|            330062794|       2601|
|175a183c35e59a5f0...|     true|   pt-br|2018-01-12 23:13:...|  true|       MARCUS|                 41|            800931027|       2684|
|23da1100a38d20dc7...|    false|   pt-br|2018-01-13 15:57:...|  true|       CLEUSA|                 91|            560342283|       2683|
|4d16176ed8c115d29...|     true|   pt-br|2018-04-06 04:10:...|  true|      RAFAELA|                 22|            703520378|       2600|
|50ad6b6855b69a84e...|    false|  

In [0]:
# Filtrando os casos que não possuem correspondência em consumers_df
missing_in_consumers_df = consumers_target_df.filter(col("customer_phone_number").isNull())

count_missing = missing_in_consumers_df.count()

# Validando se todos os casos estão com cadastro na tabela consumers
print(f"Total de clientes sem cadastro: {count_missing}")

Total de clientes sem cadastro: 311


LEFT JOIN com a tabela orders

In [0]:
# LEFT JOIN com a tabela orders
consumers_orders = ab_test_df.join(orders_df, on="customer_id", how="left")

consumers_orders.show(5)

+--------------------+---------+---------------------+----------------------+--------------------+--------------------+---------------+---------------+------------------+
|         customer_id|is_target|delivery_address_city|delivery_address_state|         merchant_id|            order_id|origin_platform|order_scheduled|order_total_amount|
+--------------------+---------+---------------------+----------------------+--------------------+--------------------+---------------+---------------+------------------+
|11eec65ca267ea72e...|     true|            SAO PAULO|                    SP|c977882744482811e...|a92eab62ae7621630...|        ANDROID|          false|              22.1|
|11eec65ca267ea72e...|     true|            SAO PAULO|                    SP|33491db6e7d1f3304...|69472729e1500d894...|        ANDROID|          false|              37.5|
|175a183c35e59a5f0...|     true| SAO BERNARDO DO C...|                    SP|fde057e2cea363bee...|6de889df7bc232e8a...|        ANDROID|          

In [0]:
# Verificando quantidade de linhas
consumers_orders_count = consumers_orders.count()
print(f"Total de linhas: {consumers_orders_count}")

Total de linhas: 2427416


In [0]:
# Agrupando e calculando a contagem e soma
result_df = consumers_orders.groupBy("customer_id", "is_target", "delivery_address_city","delivery_address_state","origin_platform", "order_scheduled") \
    .agg(
        F.count("order_id").alias("total_orders"),
        F.count("merchant_id").alias("total_merchants"),
        F.sum("order_total_amount").alias("total_amount")
    )

# Exibindo o resultado
result_df.show(5)


+--------------------+---------+---------------------+----------------------+---------------+---------------+------------+---------------+------------------+
|         customer_id|is_target|delivery_address_city|delivery_address_state|origin_platform|order_scheduled|total_orders|total_merchants|      total_amount|
+--------------------+---------+---------------------+----------------------+---------------+---------------+------------+---------------+------------------+
|0001226e517517758...|     true|       RIO DE JANEIRO|                    RJ|        ANDROID|          false|           2|              2|              86.5|
|00021cd56b6d6c980...|     true|             CAMPINAS|                    SP|        ANDROID|          false|           3|              3|              47.8|
|00021f6dc15d10418...|     true|            SAO PAULO|                    SP|        ANDROID|          false|           1|              1|             128.6|
|00021f6dc15d10418...|     true|            GUARULHO

In [0]:
# Verificando quantidade de linhas
filtered_df_count = result_df.count()
print(f"Total de linhas: {filtered_df_count}")


Total de linhas: 879195


In [0]:
# Definindo cliente para validação
target_customer_id = "00021cd56b6d6c980c3b48ca0fdf8c53cf3fea776d74ab401d02b84a5bc1bbad"

filtered_df = result_df.filter(F.col("customer_id") == target_customer_id)

# Verificando as primeiras linhas
filtered_df.limit(200).toPandas()

,customer_id,is_target,customer_name,delivery_address_city,delivery_address_state,origin_platform,order_scheduled_date,total_orders,total_merchants,total_amount
0,00021cd56b6d6c980c3b48ca0fdf8c53cf3fea776d74ab...,True,SILVIA,CAMPINAS,SP,ANDROID,None,3,3,47.8


In [0]:
# Filtrando os registros onde total_orders = 0
df_filtered = result_df.filter(col("total_orders") == 0)

# Mostrando o resultado
df_filtered.show()

+-----------+---------+---------------------+----------------------+---------------+---------------+------------+---------------+------------+
|customer_id|is_target|delivery_address_city|delivery_address_state|origin_platform|order_scheduled|total_orders|total_merchants|total_amount|
+-----------+---------+---------------------+----------------------+---------------+---------------+------------+---------------+------------+
|       null|     true|                 NULL|                  NULL|           NULL|           NULL|           0|              0|        NULL|
+-----------+---------+---------------------+----------------------+---------------+---------------+------------+---------------+------------+



In [0]:
# Removendo os registros onde total_orders = 0
result_df = result_df.filter(col("total_orders") != 0)

### ANÁLISE EXPLORATÓRIA

#### 1. ANÁLISE DE VOLUME DE PEDIDOS


**Principais Observações:**

- Os clientes contemplados na campanha de cupons apresentam, em média, um número maior de pedidos por cliente (2,89) em comparação com o grupo de controle (2,59).

- A variabilidade dos pedidos, medida pelo desvio padrão, é um pouco maior entre os clientes contemplados (3,52) em relação ao grupo de controle (3,29).

- O volume máximo de pedidos entre os clientes contemplados (157) é bastante superior ao do grupo de controle (86), apresentando uma maior dispersão. No entanto, ao analisarmos as medianas, identificamos a presença de outliers: a mediana para os clientes contemplados é de 2 pedidos, enquanto para o grupo de controle é de 1. **Concluindo que os clientes contemplados tendem a realizar mais pedidos**.

In [0]:
# Mostrar algumas estatísticas básicas do conjunto de dados
print("Contagem de clientes por grupo:")
result_df.groupBy("is_target").count().show()

Contagem de clientes por grupo:
+---------+------+
|is_target| count|
+---------+------+
|     true|489461|
|    false|389807|
+---------+------+



In [0]:
# Estatísticas: target vs controle
volume_stats = result_df.groupBy("is_target").agg(
    F.count("customer_id").alias("num_clientes"),
    F.sum("total_orders").alias("total_pedidos"),
    F.avg("total_orders").alias("media_pedidos"),
    F.stddev("total_orders").alias("desvio_padrao_pedidos"),
    F.min("total_orders").alias("min_pedidos"),
    F.max("total_orders").alias("max_pedidos"),
    F.expr("percentile_approx(total_orders, 0.5)").alias("mediana_pedidos")
).orderBy("is_target")

print("Estatísticas de volume:")
volume_stats.show()

Estatísticas de volume:
+---------+------------+-------------+------------------+---------------------+-----------+-----------+---------------+
|is_target|num_clientes|total_pedidos|     media_pedidos|desvio_padrao_pedidos|min_pedidos|max_pedidos|mediana_pedidos|
+---------+------------+-------------+------------------+---------------------+-----------+-----------+---------------+
|    false|      389763|      1010738|2.5932117722821304|   3.2874292158233844|          1|         86|              1|
|     true|      489431|      1416677|  2.89453876031555|   3.5209643016848897|          1|        157|              2|
+---------+------------+-------------+------------------+---------------------+-----------+-----------+---------------+



In [0]:
import plotly.graph_objects as go

# Coletando os dados do volume de pedidos por grupo (target vs controle)
volume_stats_df = volume_stats.toPandas()

# Calculando a porcentagem do volume de pedidos
volume_stats_df["perc_total_pedidos"] = (volume_stats_df["total_pedidos"] / volume_stats_df["total_pedidos"].sum()) * 100

#Usando os dados da tabela volume_stats_df
labels = volume_stats_df["is_target"].replace({True: 'Contemplado', False: 'Controle'}).tolist()
values = volume_stats_df["perc_total_pedidos"].tolist()
quantities = volume_stats_df["total_pedidos"].tolist()

# Ajustando os labels para incluir as quantidades
labels = [f"{label} ({quantity})" for label, quantity in zip(labels, quantities)]

# Garantindo a ordem correta das cores com códigos hexadecimais
colors = ['#FF0000' if 'Contemplado' in label else '#808080' for label in labels]

# Criando o gráfico de pizza com Plotly
fig = go.Figure(go.Pie(
    labels=labels,
    values=values,
    marker=dict(colors=colors), 
    textinfo='percent+label',
    hole=0.0 
))

# Configurando o layout
fig.update_layout(
    title="Distribuição do Volume de Pedidos por Grupo (Contemplado vs Controle)",
    showlegend=True
)

fig.show()


In [0]:
# Converter para pandas para realizar teste t
pedidos_pandas = result_df.select("is_target", "total_orders").toPandas()
target_orders = pedidos_pandas[pedidos_pandas["is_target"] == True]["total_orders"]
non_target_orders = pedidos_pandas[pedidos_pandas["is_target"] == False]["total_orders"]

# Teste t para diferença nas médias de pedidos
t_stat, p_value = stats.ttest_ind(target_orders, non_target_orders, equal_var=False)
print(f"Teste t para diferença nas médias de pedidos:")
print(f"t-statistic: {t_stat:.4f}, p-value: {p_value:.4f}")
print(f"Conclusão: {'Diferença estatisticamente significativa' if p_value < 0.05 else 'Diferença não significativa'}")

Teste t para diferença nas médias de pedidos:
t-statistic: 41.3893, p-value: 0.0000
Conclusão: Diferença estatisticamente significativa


#### 2. ANÁLISE DO VALOR


**Principais Observações:**
- Clientes que receberam os cupons possuem maior valor médio por cliente (R$ 138,38) em comparação ao grupo de controle (R$ 124,25).

- O Ticket Médio por Pedido é praticamente igual em ambos os grupos (R$ 47,92 vs R$ 47,81).

- O desvio padrão do valor é maior no grupo de controle (R$ 297,42), indicando maior dispersão.

- Embora o valor máximo dos pedidos seja superior no grupo de controle (R$ 140.338,90), trata-se de um outlier, o que reforça a importância de analisar a mediana.

- A mediana do valor gasto pelos clientes contemplados é de R$ 73,50, enquanto no grupo controle é de aproximadamente R$ 63,10, sugerindo que **os clientes contemplados tendem a gastar mais.**

In [0]:
# Estatísticas: target vs controle
valor_stats = result_df.groupBy("is_target").agg(
    F.count("customer_id").alias("num_clientes"),
    F.sum("total_amount").alias("valor_total"),
    F.avg("total_amount").alias("valor_medio_por_cliente"),
    (F.sum("total_amount") / F.sum("total_orders")).alias("ticket_medio_por_pedido"),
    F.min("total_amount").alias("valor_min"),
    F.max("total_amount").alias("valor_max"),
    F.stddev("total_amount").alias("desvio_padrao_valor"),
    F.expr("percentile_approx(total_amount, 0.5)").alias("mediana_valor")
).orderBy("is_target")

print("Estatísticas de valor (com mediana):")
valor_stats.show()

Estatísticas de valor (com mediana):
+---------+------------+--------------------+-----------------------+-----------------------+---------+------------------+-------------------+-------------+
|is_target|num_clientes|         valor_total|valor_medio_por_cliente|ticket_medio_por_pedido|valor_min|         valor_max|desvio_padrao_valor|mediana_valor|
+---------+------------+--------------------+-----------------------+-----------------------+---------+------------------+-------------------+-------------+
|    false|      389763|4.8432203490002446E7|     124.26064939463839|      47.91766361807159|      0.0|          140338.9|  297.5308806947236|         63.1|
|     true|      489431| 6.772998645000698E7|     138.38515837780398|      47.80905347514428|      0.0|37218.799999999996| 210.51717902972612|         73.5|
+---------+------------+--------------------+-----------------------+-----------------------+---------+------------------+-------------------+-------------+



In [0]:
import plotly.graph_objects as go
import pandas as pd

# Coletando os dados do volume de pedidos por grupo (target vs controle)
valor_stats_df = valor_stats.toPandas()

# Calculando a porcentagem do valor total
valor_stats_df["perc_total_valor"] = (valor_stats_df["valor_total"] / valor_stats_df["valor_total"].sum()) * 100

# Usando os dados da tabela valor_stats_df
labels = valor_stats_df["is_target"].replace({True: 'Contemplado', False: 'Controle'}).tolist()
values = valor_stats_df["perc_total_valor"].tolist()
quantities = valor_stats_df["valor_total"].tolist()

# Ajustando os labels para incluir os valores totais
labels = [f"{label} (R$ {quantity:,.2f})" for label, quantity in zip(labels, quantities)]

# Garantindo a ordem correta das cores
colors = ['#FF0000' if 'Contemplado' in label else '#808080' for label in labels]

# Criando o gráfico de pizza com Plotly
fig = go.Figure(go.Pie(
    labels=labels,
    values=values,
    marker=dict(colors=colors), 
    textinfo='percent+label',
    hole=0.0
))

# Configurando o layout
fig.update_layout(
    title="Distribuição do Valor Total por Grupo (Contemplado vs Controle)",
    showlegend=True
)

fig.show()

In [0]:
# Teste t para diferença nas médias de valor
valor_pandas = result_df.select("is_target", "total_amount").toPandas()
target_amount = valor_pandas[valor_pandas["is_target"] == True]["total_amount"]
non_target_amount = valor_pandas[valor_pandas["is_target"] == False]["total_amount"]

t_stat_valor, p_value_valor = stats.ttest_ind(target_amount, non_target_amount, equal_var=False)
print(f"Teste t para diferença nas médias de valor:")
print(f"t-statistic: {t_stat_valor:.4f}, p-value: {p_value_valor:.4f}")
print(f"Conclusão: {'Diferença estatisticamente significativa' if p_value_valor < 0.05 else 'Diferença não significativa'}")

Teste t para diferença nas médias de valor:
t-statistic: 25.0785, p-value: 0.0000
Conclusão: Diferença estatisticamente significativa


#### 3. ANÁLISE DE MERCHANTS


**Principais Observações:**

- Clientes que receberam os cupons possuem maior diversidade de merchants por cliente (2,89) em comparação aos do grupo de controle (2,59).

- O desvio padrão é maior para os clientes que foram contemplados pelos cupons (3,52), indicando maior variação na quantidade de merchants visitados.

- O máximo de merchants visitados por um único cliente é maior entre os clientes que receberam os cupons (157), enquanto o máximo entre o grupo de controle é 86.

In [0]:
# Estatísticas: target vs controle
merchants_stats = result_df.groupBy("is_target").agg(
    count("customer_id").alias("num_clientes"),
    sum("total_merchants").alias("total_merchants"),
    avg("total_merchants").alias("media_merchants_por_cliente"),
    stddev("total_merchants").alias("desvio_padrao_merchants"),
    min("total_merchants").alias("min_merchants"),
    max("total_merchants").alias("max_merchants")
).orderBy("is_target")

print("Estatísticas de diversidade de merchants:")
merchants_stats.show()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:528)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:633)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:656)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

In [0]:
# Calcular a proporção de merchants por pedidos para cada grupo

df = result_df.withColumn(
    "pedidos_por_merchants", 
    F.when(F.col("total_orders") > 0, F.col("total_orders") / F.col("total_merchants")).otherwise(0)
)

pedidos_por_merchants_stats = df.groupBy("is_target").agg(
    F.avg("pedidos_por_merchants").alias("media_pedidos_por_merchants"),
    F.stddev("pedidos_por_merchants").alias("desvio_padrao_pedidos_por_merchants")
).orderBy("is_target")

print("Proporção de pedidos por merchants:")



Proporção de pedidos por merchants:
+---------+---------------------------+-----------------------------------+
|is_target|media_pedidos_por_merchants|desvio_padrao_pedidos_por_merchants|
+---------+---------------------------+-----------------------------------+
|    false|                        1.0|                                0.0|
|     true|                        1.0|                                0.0|
+---------+---------------------------+-----------------------------------+



In [0]:
pedidos_por_merchants_stats.show()

+---------+---------------------------+-----------------------------------+
|is_target|media_pedidos_por_merchants|desvio_padrao_pedidos_por_merchants|
+---------+---------------------------+-----------------------------------+
|    false|                        1.0|                                0.0|
|     true|                        1.0|                                0.0|
+---------+---------------------------+-----------------------------------+



#### 4. ANÁLISE GEOGRÁFICA


In [0]:
# Distribuição geográfica por estado
geo_state_dist = result_df.groupBy("is_target", "delivery_address_state").count().orderBy("is_target", "delivery_address_state")

In [0]:
# Calcular percentuais dentro de cada grupo
window_spec = Window.partitionBy("is_target")
geo_state_dist = geo_state_dist.withColumn("percentual", 
                                          col("count") / sum("count").over(window_spec) * 100)

print("Distribuição geográfica por estado:")
geo_state_dist.show(30)  # Mostrar mais linhas para ver múltiplos estados

Distribuição geográfica por estado:
+---------+----------------------+------+--------------------+
|is_target|delivery_address_state| count|          percentual|
+---------+----------------------+------+--------------------+
|    false|                    AC|     1|2.565372094395431...|
|    false|                    AL|  2033|  0.5215401467905912|
|    false|                    AM|  4723|  1.2116252401829624|
|    false|                    BA| 11338|    2.90861888062554|
|    false|                    CE| 14049|   3.604091255416142|
|    false|                    DF| 13699|  3.5143032321123013|
|    false|                    ES|  8040|  2.0625591638939267|
|    false|                    GO|  3450|  0.8850533725664239|
|    false|                    MA|   896| 0.22985733965783067|
|    false|                    MG| 21764|   5.583275826242217|
|    false|                    MS|  1826|  0.4684369444366058|
|    false|                    MT|   437| 0.11210676052508035|
|    false|        

In [0]:
# Distribuição por cidade (top 10 para cada grupo)
geo_city_dist = result_df.groupBy("is_target", "delivery_address_city").count()

In [0]:
# janela para soma por grupo
window_spec = Window.partitionBy("is_target")

# calcular percentual
geo_city_dist = geo_city_dist.withColumn(
    "percentual",
    (F.col("count") / F.sum(F.col("count")).over(window_spec)) * 100
)

# janela para rank
rank_window = Window.partitionBy("is_target").orderBy(F.col("count").desc())

# calcular rank
geo_city_dist = geo_city_dist.withColumn("rank", F.rank().over(rank_window))

print("Top 10 cidades por grupo:")
geo_city_dist.filter(F.col("rank") <= 10).orderBy("is_target", "rank").show(20)


Top 10 cidades por grupo:
+---------+---------------------+------+------------------+----+
|is_target|delivery_address_city| count|        percentual|rank|
+---------+---------------------+------+------------------+----+
|    false|            SAO PAULO|104240|  26.7444575293191|   1|
|    false|       RIO DE JANEIRO| 62090|15.930193476548569|   2|
|    false|            FORTALEZA| 14014|3.5955183021477155|   3|
|    false|       BELO HORIZONTE| 13888|3.5631909647657682|   4|
|    false|             BRASILIA| 13699| 3.514699958692847|   5|
|    false|               RECIFE| 13255| 3.400784579346936|   6|
|    false|             CURITIBA| 12206| 3.131646667333739|   7|
|    false|             SALVADOR| 10868|2.7883611322778203|   8|
|    false|         PORTO ALEGRE| 10604|2.7206276634775493|   9|
|    false|             CAMPINAS|  7808| 2.003268653001953|  10|
|     true|            SAO PAULO|130597|26.683434437132096|   1|
|     true|       RIO DE JANEIRO| 78872| 16.11503970937681|   2|

In [0]:
# Convertendo para DataFrame do Pandas
geo_df = geo_city_dist.toPandas()

# Separando os dois grupos
geo_df_true = geo_df[geo_df["is_target"] == True].nlargest(10, "percentual")
geo_df_false = geo_df[geo_df["is_target"] == False].nlargest(10, "percentual")

# Garantindo que as cidades estejam alinhadas e ordenadas
cities = list(set(geo_df_true["delivery_address_city"].tolist() + geo_df_false["delivery_address_city"].tolist()))
geo_df_true = geo_df_true.set_index("delivery_address_city").reindex(cities).fillna(0)
geo_df_false = geo_df_false.set_index("delivery_address_city").reindex(cities).fillna(0)

# Ordenando as cidades pela soma dos percentuais
geo_df_combined = geo_df_true["percentual"] + geo_df_false["percentual"]
cities = geo_df_combined.sort_values(ascending=True).index.tolist()
geo_df_true = geo_df_true.loc[cities]
geo_df_false = geo_df_false.loc[cities]

# Criando o gráfico com Plotly
fig = go.Figure()

fig.add_trace(go.Bar(
    y=cities,
    x=geo_df_true["percentual"],
    name="Clientes Contemplados",
    orientation="h",
    marker_color="#e15759",
    text=[f'{val:.2f}%' for val in geo_df_true["percentual"]],
    textposition="auto"
))

fig.add_trace(go.Bar(
    y=cities,
    x=geo_df_false["percentual"],
    name="Clientes Controle",
    orientation="h",
    marker_color="#bab0ac",
    text=[f'{val:.2f}%' for val in geo_df_false["percentual"]],
    textposition="auto"
))

fig.update_layout(
    title="Top 10 Cidades por Grupo (Clientes Contemplados vs Controle)",
    xaxis_title="Percentual (%)",
    yaxis_title="Cidades",
    barmode="group",
    template="simple_white",
    height=600,
    width=800
)

fig.show()

#### 5. ANÁLISE DE RETENÇÃO

**Principais Observações:**

- Os clientes do grupo que recebeu os cupons apresentam uma taxa de retenção significativamente maior (53,35%) em comparação com o grupo de controle (44,80%), indicando maior fidelidade.

- O número total de pedidos realizados pelo grupo contemplado (1.416.677) é consideravelmente maior que o do grupo de controle (1.010.738), sugerindo maior engajamento dos clientes contemplados.

- O valor total gasto pelos clientes contemplados (aproximadamente 67,7 milhões) supera o valor gasto pelo grupo de controle (cerca de 48,4 milhões), mostrando um impacto positivo no faturamento.

Sendo assim, os clientes **contemplados pela campanha de aumento são mais engajados e geram maior valor para o negócio**, refletindo em melhores métricas de retenção e consumo.

In [0]:
df = result_df.withColumn("is_retained", F.when(F.col("total_orders") > 1, 1).otherwise(0))

In [0]:
retention_df = df.groupBy("is_target") \
    .agg(
        F.countDistinct("customer_id").alias("total_customers"),
        F.sum("is_retained").alias("retained_customers")
    ) \
    .withColumn("retention_rate", F.col("retained_customers") / F.col("total_customers"))


In [0]:
# Calculando retenção por grupo de is_target
retention_analysis = (
    df.groupBy("is_target")
    .agg(
        F.count("customer_id").alias("total_customers"),
        F.sum(F.when(F.col("is_retained") == 1, 1).otherwise(0)).alias("retained_customers"),
        F.avg("is_retained").alias("retention_rate"),
        F.sum("total_orders").alias("total_orders"),
        F.sum("total_amount").alias("total_amount")
    )
)

retention_analysis.show()

+---------+---------------+------------------+------------------+------------+--------------------+
|is_target|total_customers|retained_customers|    retention_rate|total_orders|        total_amount|
+---------+---------------+------------------+------------------+------------+--------------------+
|     true|         489431|            261117|0.5335113631952205|     1416677| 6.772998645000698E7|
|    false|         389763|            174596| 0.447954269645913|     1010738|4.8432203490002446E7|
+---------+---------------+------------------+------------------+------------+--------------------+



### **CLUSTERIZAÇÃO**

KMeans: O modelo KMeans foi treinado e aplicado para agrupar os dados em dois clusters.

- Agregação por Cluster: Foram calculadas as somas de pedidos, valores e retenções para cada cluster.

- O Cluster 0 apresentou uma vantagem significativa para a ação. Por esse motivo, os próximos passos incluem aprofundar a análise desse cluster, verificando a possibilidade de segmentá-lo ainda mais para otimizar os indicadores. Para os próximoas passos, será interessante incluir novas variáveis no modelo e criar uma próxima campanha de cupons para realizar testes A/B, além de análises temporais para avaliar o desempenho ao longo do tempo.

In [0]:
# Filtrar só onde is_target == True
target_retention_df = df.filter(F.col("is_target") == True)

In [0]:
target_retention_df = target_retention_df[["customer_id", "is_target", "delivery_address_city", "delivery_address_state", "origin_platform", "order_scheduled", "total_orders", "total_merchants", "total_amount", "is_retained"]]

In [0]:

#Transformar booleanos para inteiro (0/1)
bool_cols = ['is_target', 'order_scheduled', 'is_retained']
for col in bool_cols:
    target_retention_df = target_retention_df.withColumn(col, target_retention_df[col].cast('int'))

# Colunas categóricas
cat_cols = ['delivery_address_city', 'delivery_address_state', 'origin_platform']
index_output_cols = [c + "_idx" for c in cat_cols]
encoder_output_cols = [c + "_vec" for c in cat_cols]

indexers = [StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid="keep") for c in cat_cols]

# OneHotEncoder para todas as colunas indexadas
encoder = OneHotEncoder(inputCols=index_output_cols, outputCols=encoder_output_cols)

# Pipeline
pipeline = Pipeline(stages=indexers + [encoder])
target_retention_df = pipeline.fit(target_retention_df).transform(target_retention_df)

# Montar vetor de features
feature_cols = ['total_orders', 'total_merchants', 'total_amount'] + bool_cols + encoder_output_cols
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
target_retention_df = assembler.transform(target_retention_df)

# KMeans (distância Euclidiana)
k = 2
kmeans = KMeans(featuresCol="features", predictionCol="cluster", k=k, seed=42)
model = kmeans.fit(target_retention_df)
target_retention_df = model.transform(target_retention_df)

# Agregar por cluster
agg_df = target_retention_df.groupBy('cluster').agg(
    spark_sum('total_orders').alias('total_orders_sum'),
    spark_sum('total_amount').alias('total_amount_sum'),
    spark_sum('is_retained').alias('is_retained_sum')
)

agg_df.show()


In [0]:
agg_stats = target_retention_df.groupBy('cluster').agg(
    avg('total_orders').alias('avg_total_orders'),
    avg('total_merchants').alias('avg_total_merchants'),
    avg('total_amount').alias('avg_total_amount'),
    avg('is_target').alias('avg_is_target'),
    avg('order_scheduled').alias('avg_order_scheduled'),
    avg('is_retained').alias('avg_is_retained')
)

agg_stats.show()


+-------+------------------+-------------------+------------------+-------------+--------------------+------------------+
|cluster|  avg_total_orders|avg_total_merchants|  avg_total_amount|avg_is_target| avg_order_scheduled|   avg_is_retained|
+-------+------------------+-------------------+------------------+-------------+--------------------+------------------+
|      1|2.3144596691133335| 2.3144596691133335|100.05501232025068|          1.0|3.020307767188593E-4|0.5041567257256344|
|      0|12.033033238626638| 12.033033238626638|   742.23355321261|          1.0| 5.47701365830281E-4|0.9959607024270016|
+-------+------------------+-------------------+------------------+-------------+--------------------+------------------+



In [0]:
# Criando chave combinada para avaliar grupos de maior impacto

target_retention_df = target_retention_df.withColumn(
    "combined_key",
    F.concat_ws("_", 
                F.col("delivery_address_city"), 
                F.col("delivery_address_state"), 
                F.col("origin_platform"))
)

target_retention_df.show(truncate=False)

+----------------------------------------------------------------+---------+-----------------------+----------------------+---------------+---------------+------------+---------------+------------------+-----------+-------------------------+--------------------------+-------------------+-------------------------+--------------------------+-------------------+-----------------------------------------------------------------------------+-------+------------------------------+
|customer_id                                                     |is_target|delivery_address_city  |delivery_address_state|origin_platform|order_scheduled|total_orders|total_merchants|total_amount      |is_retained|delivery_address_city_idx|delivery_address_state_idx|origin_platform_idx|delivery_address_city_vec|delivery_address_state_vec|origin_platform_vec|features                                                                     |cluster|combined_key                  |
+-----------------------------------------

In [0]:
target_retention_df.groupBy('cluster', 'combined_key').count().orderBy('count', ascending=False).show()

+-------+--------------------+-----+
|cluster|        combined_key|count|
+-------+--------------------+-----+
|      1|    SAO PAULO_SP_IOS|54104|
|      1|SAO PAULO_SP_ANDROID|43803|
|      1|RIO DE JANEIRO_RJ...|31230|
|      1|RIO DE JANEIRO_RJ...|29223|
|      1|SAO PAULO_SP_DESKTOP|21282|
|      1|RIO DE JANEIRO_RJ...|10432|
|      1|    FORTALEZA_CE_IOS| 7945|
|      1|BELO HORIZONTE_MG...| 7647|
|      1|     BRASILIA_DF_IOS| 6811|
|      1|BELO HORIZONTE_MG...| 6800|
|      1|   RECIFE_PE_ANDROID| 6686|
|      1|FORTALEZA_CE_ANDROID| 6544|
|      1|       RECIFE_PE_IOS| 6469|
|      0|    SAO PAULO_SP_IOS| 6446|
|      1| BRASILIA_DF_ANDROID| 6310|
|      1| CURITIBA_PR_ANDROID| 6116|
|      1|     CURITIBA_PR_IOS| 6052|
|      1| SALVADOR_BA_ANDROID| 5599|
|      1|     SALVADOR_BA_IOS| 5493|
|      1|PORTO ALEGRE_RS_A...| 5312|
+-------+--------------------+-----+
only showing top 20 rows



In [0]:
target_retention_df.groupBy('cluster', 'combined_key') \
    .agg(
        F.count('*').alias('total'),
        F.sum(F.col('is_retained')).alias('retained'),
        (F.sum(F.col('is_retained')) / F.count('*')).alias('retention_rate')
    ) \
    .orderBy('retention_rate', ascending=False) \
    .show()


+-------+--------------------+-----+--------+--------------+
|cluster|        combined_key|total|retained|retention_rate|
+-------+--------------------+-----+--------+--------------+
|      0|  GOIANIA_GO_ANDROID|   45|      45|           1.0|
|      0|  NITEROI_RJ_DESKTOP|   56|      56|           1.0|
|      0|SAO GONCALO_RJ_AN...|    2|       2|           1.0|
|      0|  JANDIRA_SP_ANDROID|   10|      10|           1.0|
|      0|PARNAMIRIM_RN_AND...|    5|       5|           1.0|
|      0|     CONTAGEM_MG_IOS|    9|       9|           1.0|
|      0|SAO JOSE DO RIO P...|   68|      68|           1.0|
|      0|  DIADEMA_SP_DESKTOP|    5|       5|           1.0|
|      0|GOIANIA_GO_WINDOW...|    1|       1|           1.0|
|      0|SANTO ANDRE_SP_AN...|   63|      63|           1.0|
|      0|  VITORIA_ES_ANDROID|  116|     116|           1.0|
|      0|SAO BERNARDO DO C...|   11|      11|           1.0|
|      0|CAMPINA GRANDE_PB...|    7|       7|           1.0|
|      0|   RECIFE_PE_AN

In [0]:
target_retention_df.groupBy('cluster', 'delivery_address_city').count().show()
target_retention_df.groupBy('cluster', 'delivery_address_state').count().show()
target_retention_df.groupBy('cluster', 'origin_platform').count().show()

+-------+---------------------+-----+
|cluster|delivery_address_city|count|
+-------+---------------------+-----+
|      0|             SAO JOSE|   12|
|      1| SAO BERNARDO DO C...| 3762|
|      0|       BELO HORIZONTE|  449|
|      1|           SEROPEDICA|    1|
|      1|              CUBATAO|    3|
|      0|      TABOAO DA SERRA|   17|
|      1|              FORMOSA|    9|
|      1|       RIO DE JANEIRO|71707|
|      1|               SOBRAL|   16|
|      0|              NITEROI|  546|
|      0| SAO BERNARDO DO C...|  135|
|      0|       CAMPINA GRANDE|   24|
|      1|              TAUBATE|  372|
|      0|           CAMARAGIBE|    1|
|      1|             PAULISTA|   91|
|      1|                BELEM| 2987|
|      1| SAO JOSE DO RIO P...| 5093|
|      0|   SAO CAETANO DO SUL|  140|
|      0|        VOLTA REDONDA|   27|
|      0|      DUQUE DE CAXIAS|   29|
+-------+---------------------+-----+
only showing top 20 rows

+-------+----------------------+-----+
|cluster|delivery_addre